In [1]:
import tensorflow as tf

In [2]:
daily_sales_number = [21, 22, -108, 31, -1, 32, 34, 31]

In [3]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_number)
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [4]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [5]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [6]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [7]:
tf_dataset = tf_dataset.map(lambda x: x*5)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

105
110
155
160
170
155


In [8]:
tf_dataset = tf_dataset.shuffle(4)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

110
170
105
155
160
155


In [9]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[160 155]
[105 155]
[110 170]


In [10]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_number)

tf_dataset = tf_dataset.filter(lambda x : x>0).map(lambda y:y*5).shuffle(2).batch(2)

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[105 155]
[160 170]
[110 155]


In [11]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

for file in images_ds.take(5):
    print(file.numpy())

b'images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg'
b'images\\cat\\7 Foods Your Cat Can_t Eat.jpg'
b'images\\cat\\A cat appears to have caught the....jpg'
b'images\\cat\\Adopt-A-Cat Month\xc2\xae - American Humane....jpg'
b'images\\cat\\All About Your Cat_s Tongue.jpg'


In [12]:
images_ds = images_ds.shuffle(200)

for file in images_ds.take(3):
    print(file.numpy())

b'images\\cat\\What to do if your cat is marking....jpg'
b'images\\dog\\The 20 Best Dog Breeds for Runners....jpg'
b'images\\dog\\Hong Kong dog causes panic \xe2\x80\x93 but here_s....jpg'


In [13]:
class_names = ['cat', 'dog']

In [14]:
image_count = len(images_ds)
image_count

130

In [15]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [16]:
len(train_ds)

104

In [17]:
len(test_ds)

26

In [18]:
s = 'images\\cat\\All About Your Cat_s Tongue.jpg'

s.split('\\')[-2]

'cat'

In [19]:
def get_label(file_path):
    import os
    return tf.strings.split(file_path, os.path.sep)[-2]

In [20]:
def process_image(file_path):
    label = get_label(file_path)

    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])

    return img, label

In [21]:
for t in train_ds.take(4):
    print(t.numpy())

b'images\\dog\\dog existed at the end of the Ice Age_yythkg....jpg'
b'images\\dog\\The Best Dogs of BBC Earth _ Top 5....jpg'
b'images\\cat\\Home_ sweet home_ How to bring an....jpg'
b'images\\dog\\11 Things Humans Do That Dogs Hate.jpg'


In [22]:
train_ds = train_ds.map(process_image)
for img, label in train_ds.take(3):
    print('image : ', img)
    print('label : ', label)

image :  tf.Tensor(
[[[ 36.75  35.75  41.75]
  [ 35.25  35.25  43.25]
  [ 34.    34.    42.  ]
  ...
  [ 63.    49.    48.  ]
  [ 66.25  51.5   48.5 ]
  [ 85.5   61.5   49.5 ]]

 [[ 35.25  35.25  43.25]
  [ 36.    36.    44.  ]
  [ 35.    35.    43.  ]
  ...
  [ 69.    51.75  45.5 ]
  [ 73.75  54.75  48.5 ]
  [ 93.25  67.25  53.75]]

 [[ 34.75  34.75  42.75]
  [ 34.25  34.25  42.25]
  [ 35.75  35.75  43.75]
  ...
  [ 77.25  58.    50.5 ]
  [ 89.5   66.5   52.5 ]
  [ 97.    69.75  53.5 ]]

 ...

 [[145.25 129.25 113.25]
  [143.75 131.75 109.75]
  [145.75 133.75 111.75]
  ...
  [110.75 103.75  85.75]
  [115.25 108.25  92.25]
  [112.   103.    86.  ]]

 [[143.25 127.25 111.25]
  [141.25 128.25 109.25]
  [146.   131.   110.  ]
  ...
  [110.75 103.75  87.75]
  [106.75  99.75  83.75]
  [108.5  100.75  87.  ]]

 [[147.75 131.75 115.75]
  [150.   135.   114.  ]
  [144.25 129.25 108.25]
  ...
  [109.75 103.75  91.75]
  [105.    97.5   84.25]
  [107.   101.    87.  ]]], shape=(128, 128, 3), dtyp

In [23]:
def scale(image, label):
    return image/255, label

In [24]:
train_ds = train_ds.map(scale)
for image, label in train_ds.take(5):
    print('*******Image: ', image.numpy()[0][0])
    print('*******Label: ', label.numpy())

*******Image:  [0.99607843 0.99607843 0.99607843]
*******Label:  b'dog'
*******Image:  [0.4059283  0.24524356 0.13610217 1.        ]
*******Label:  b'dog'
*******Image:  [0.50351036 0.46697664 0.25398403]
*******Label:  b'dog'
*******Image:  [0.860769   0.79018074 0.6960631 ]
*******Label:  b'dog'
*******Image:  [0.3577304 0.6239124 0.4327638]
*******Label:  b'dog'
